In [1]:
'''
Creates files for 3' end for RNAPII position as CDF
KS test caluclations
'''

'\nKS test caluclations\n'

In [2]:
import os
import re

import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import mygene
import scipy

from plotnine import *
import warnings
warnings.filterwarnings('ignore')

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # export pdfs with editable font types in Illustrator

In [4]:
# bed position for 3' end of LRS reads
bedlist1 = [
    '/home/mp758/scratch60/SF3B1_PRC2_1/April_2022/MUT3/MUT3.intersect.transcript-bed.waos.ratio.bed',
    
    '/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/MUT2/MUT2.intersect.transcript-bed.waos.ratio.bed', 
    '/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/WT1/WT1.intersect.transcript-bed.waos.ratio.bed', 
    '/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/WT2/WT2.intersect.transcript-bed.waos.ratio.bed'
            ]
    
    
    
    

In [5]:
# function to generate relative position across gene body
# separately for each of the clusters (defined by RNAPII ChiP-seq). But can be run for one bed
def Threeprimeposition(bed):
    # read gtf of all expressed genes
    gtf1 = pd.read_csv('/home/mp758/scratch60/GenomeBuilds_scratch60/Chiazzi/GSE148433_human.refseq.major.TR.gtf',
            sep = "\t", skiprows=3, index_col=False,
            names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I']
                  )
    gtf1['gene_id'] = gtf1['I'].str.split(";").str[0].str.split(" ").str[1].str.strip("\"") # split gene name from 9th column
    gtf1['transcript_id'] = gtf1['I'].str.split(";").str[1].str.split(" ").str[2].str.strip("\"") # split transcript_id name from 9th column
    namea, nameb = os.path.split(bed)
    namec = nameb.split(".")[0]
    bed = pd.read_csv(bed, 
                      delimiter = '\t', index_col = False,
                      names =  ['chr', 'start', 'end', 'name', 'score', 'strand','chrb', 'startb', 'endb', 'nameb', 'scoreb', 'strandb',"A13","A14","A15","ratio" ])
    # use aggregate function to count
    agg_func_math = {
        'ratio':
        ['mean', 'median', 'min', 'max', 'std', 'var', 'count']
        }
    three_prime_position = bed.groupby(['name']).agg(agg_func_math).round(2)
    three_prime_position.columns = ['mean', 'median', 'min', 'max', 'std', 'var', 'count']
    three_prime_position = three_prime_position[three_prime_position['count'] > 10]
    three_prime_position  = three_prime_position.reset_index()
    three_prime_position = three_prime_position.rename(columns = {'name':'transcript_id'})
    
    bed1=pd.read_csv('/gpfs/ysm/scratch60/pillai/pb694/SF3B1/Feb_2022/ChIP_clusteranalysis/Cluster1.bed',
                 sep = "\t", skiprows=3, index_col=False,
            names = ['chr', 'start', 'end', 'gene_id']
                )
    bed2=pd.read_csv('/gpfs/ysm/scratch60/pillai/pb694/SF3B1/Feb_2022/ChIP_clusteranalysis/Cluster2.bed',
                 sep = "\t", skiprows=3, index_col=False,
            names = ['chr', 'start', 'end', 'gene_id']
                )
    bed3=pd.read_csv('/gpfs/ysm/scratch60/pillai/pb694/SF3B1/Feb_2022/ChIP_clusteranalysis/Cluster3.bed',
                 sep = "\t", skiprows=3, index_col=False,
            names = ['chr', 'start', 'end', 'gene_id']
                )
    gtf1_cluster1 = gtf1.merge(bed1, on='gene_id', how = "inner")
    gtf1_cluster2 = gtf1.merge(bed2, on='gene_id', how = "inner")
    gtf1_cluster3 = gtf1.merge(bed3, on='gene_id', how = "inner")
    
    Cluster1_three_prime_position = gtf1_cluster1.merge(three_prime_position, on = 'transcript_id', how = 'inner')
    Cluster2_three_prime_position = gtf1_cluster2.merge(three_prime_position, on = 'transcript_id', how = 'inner')
    Cluster3_three_prime_position = gtf1_cluster3.merge(three_prime_position, on = 'transcript_id', how = 'inner')
    
    Cluster1_three_prime_position = Cluster1_three_prime_position [['transcript_id',
       'chr', 'start', 'end', 'mean', 'median', 'min', 'max', 'std', 'var',
       'count']]
    Cluster2_three_prime_position = Cluster2_three_prime_position [['transcript_id',
       'chr', 'start', 'end', 'mean', 'median', 'min', 'max', 'std', 'var',
       'count']]
    Cluster3_three_prime_position = Cluster3_three_prime_position [['transcript_id',
       'chr', 'start', 'end', 'mean', 'median', 'min', 'max', 'std', 'var',
       'count']]
    
    Cluster1_three_prime_position.to_csv(namea+'/'+namec+"_Cluster1_3primeposition.txt", sep = "\t", header = True, index=False)
    Cluster2_three_prime_position.to_csv(namea+'/'+namec+"_Cluster2_3primeposition.txt", sep = "\t", header = True, index=False)
    Cluster3_three_prime_position.to_csv(namea+'/'+namec+"_Cluster3_3primeposition.txt", sep = "\t", header = True, index=False)


In [ ]:
for bed in bedlist2:
    Threeprimeposition(bed)
    

In [49]:
# Running KS test 

def KS_2samp():
    folder = '/home/mp758/scratch60/SF3B1_PRC2_1/SF3B1_LRS/porechop/3primeposition/combined/'
    for i in range(1,4):
        
        data1 = pd.read_csv(folder+"WT1-2"+"_Cluster"+str(i)+"_3primeposition.txt",
                    sep = "\t",  index_col=False
                           )
        data2 = pd.read_csv(folder+"MUT2-3"+"_Cluster"+str(i)+"_3primeposition.txt",
                    sep = "\t",  index_col=False
                           )
        a = "WT1-2"+"_Cluster"+str(i)
        b = "MUT2-3"+"_Cluster"+str(i)
        print(a)
        print(b)
        #print(data1.head(2))
        #print(data2.head(2))
        with open(folder+"/KS_Cluster1-3.txt", 'a') as f:
            f.write(a + "\t" + b + "\t" + str(scipy.stats.ks_2samp(data1['mean'], data2['mean'])) + "\n")
        print(scipy.stats.ks_2samp(data1['mean'], data2['mean']))
            

KS_2samp()